In [1]:
# !pip install lxml
# !pip install html5lib

In [1]:
import requests
import os
import csv
from bs4 import BeautifulSoup
from datetime import datetime
import pandas as pd
import html5lib

In [21]:
base_url = "https://www.sec.gov"
url = "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&filenum=028-04545&owner=exclude&count=40"
xmls = []
page = requests.get(url)
soup = BeautifulSoup(page.text, "lxml")
for tr in soup.find("table",{"class":"tableFile2"}).find_all("tr"):
    first_td = tr.find("td")
    if first_td:
        if (first_td.text == "13F-HR"): # don't want amendments
            for a in tr.find_all("a", href=True):
                link= "/".join(a["href"].split("/")[0:6])
                xmls.append(base_url + link + "/xslForm13F_X01/form13fInfoTable.xml")

print(xmls)

['https://www.sec.gov/Archives/edgar/data/1067983/000095012317007953/xslForm13F_X01/form13fInfoTable.xml', 'https://www.sec.gov/Archives/edgar/data/1067983/000095012317005259/xslForm13F_X01/form13fInfoTable.xml', 'https://www.sec.gov/Archives/edgar/data/1067983/000095012317002417/xslForm13F_X01/form13fInfoTable.xml', 'https://www.sec.gov/Archives/edgar/data/1067983/000095012316022377/xslForm13F_X01/form13fInfoTable.xml', 'https://www.sec.gov/Archives/edgar/data/1067983/000095012316020120/xslForm13F_X01/form13fInfoTable.xml', 'https://www.sec.gov/Archives/edgar/data/1067983/000095012316017295/xslForm13F_X01/form13fInfoTable.xml', 'https://www.sec.gov/Archives/edgar/data/1067983/000095012316015025/xslForm13F_X01/form13fInfoTable.xml', 'https://www.sec.gov/Archives/edgar/data/1067983/000095012315011992/xslForm13F_X01/form13fInfoTable.xml', 'https://www.sec.gov/Archives/edgar/data/1067983/000095012315009404/xslForm13F_X01/form13fInfoTable.xml', 'https://www.sec.gov/Archives/edgar/data/1067

In [22]:
# BEGIN IMPORTS
import xml.etree.ElementTree as ET
# END IMPORTS

In [23]:
def xml2df(xml_data):
    root = ET.XML(xml_data) # element tree
    all_records = [] #This is our record list which we will convert into a dataframe
    for i, child in enumerate(root): #Begin looping through our root tree
        record = {} #Place holder for our record
        for subchild in child: #iterate through the subchildren to user-agent, Ex: ID, String, Description.
            record[subchild.tag] = subchild.text #Extract the text create a new dictionary key, value pair
            all_records.append(record) #Append this record to all_records.
    return pd.DataFrame(all_records) #return records as DataFrame

In [39]:
for myxml in xmls[0:1]:
    #print(myxml)
    page = requests.get(myxml)
    soup = BeautifulSoup(page.text, "lxml")
    table = soup.find("table",{"summary":"Form 13F-NT Header Information"})

    
    #column_headers = ['ID','Available Quota', 'Live Weight Pounds', 'Price', 'Date Posted']
    records = []
    for idy, row in enumerate(table.findAll('tr')):
        if idy == 0:
            continue
        cols = row.findAll('td')
        record = {}
        for idx, col in enumerate(cols):
            record[idx] = col.text.strip()
            #print(col.text.strip())
        records.append(record)
        #print(records)

    print(records)
  

[{0: '', 1: '', 2: '', 3: 'VALUE', 4: 'SHRS OR', 5: 'SH/', 6: 'PUT/', 7: 'INVESTMENT', 8: 'OTHER', 9: 'VOTING AUTHORITY'}, {0: 'NAME OF ISSUER', 1: 'TITLE OF CLASS', 2: 'CUSIP', 3: '(x$1000)', 4: 'PRN AMT', 5: 'PRN', 6: 'CALL', 7: 'DISCRETION', 8: 'MANAGER', 9: 'SOLE', 10: 'SHARED', 11: 'NONE'}, {0: 'AMERICAN AIRLS GROUP INC', 1: 'COM', 2: '02376R102', 3: '1,094,057', 4: '21,742,000', 5: 'SH', 6: '', 7: 'DFND', 8: '4', 9: '21,742,000', 10: '0', 11: '0'}, {0: 'AMERICAN AIRLS GROUP INC', 1: 'COM', 2: '02376R102', 3: '1,182,429', 4: '23,498,200', 5: 'SH', 6: '', 7: 'DFND', 8: '4,8,11', 9: '23,498,200', 10: '0', 11: '0'}, {0: 'AMERICAN AIRLS GROUP INC', 1: 'COM', 2: '02376R102', 3: '88,553', 4: '1,759,800', 5: 'SH', 6: '', 7: 'DFND', 8: '4,11', 9: '1,759,800', 10: '0', 11: '0'}, {0: 'AMERICAN EXPRESS CO', 1: 'COM', 2: '025816109', 3: '164,448', 4: '1,952,142', 5: 'SH', 6: '', 7: 'DFND', 8: '4', 9: '1,952,142', 10: '0', 11: '0'}, {0: 'AMERICAN EXPRESS CO', 1: 'COM', 2: '025816109', 3: '1,45

In [41]:
df = pd.DataFrame.from_dict(records)
df.head(20)
                   

,0,1,2,3,4,5,6,7,8,9,10,11
0,,,,VALUE,SHRS OR,SH/,PUT/,INVESTMENT,OTHER,VOTING AUTHORITY,NaN,NaN
1,NAME OF ISSUER,TITLE OF CLASS,CUSIP,(x$1000),PRN AMT,PRN,CALL,DISCRETION,MANAGER,SOLE,SHARED,NONE
2,AMERICAN AIRLS GROUP INC,COM,02376R102,"1,094,057","21,742,000",SH,,DFND,4,"21,742,000",0,0
3,AMERICAN AIRLS GROUP INC,COM,02376R102,"1,182,429","23,498,200",SH,,DFND,"4,8,11","23,498,200",0,0
4,AMERICAN AIRLS GROUP INC,COM,02376R102,"88,553","1,759,800",SH,,DFND,"4,11","1,759,800",0,0
5,AMERICAN EXPRESS CO,COM,025816109,"164,448","1,952,142",SH,,DFND,4,"1,952,142",0,0
6,AMERICAN EXPRESS CO,COM,025816109,"1,451,068","17,225,400",SH,,DFND,"4,5","17,225,400",0,0
7,AMERICAN EXPRESS CO,COM,025816109,"70,747","839,832",SH,,DFND,"4,7","839,832",0,0
8,AMERICAN EXPRESS CO,COM,025816109,"173,290","2,057,100",SH,,DFND,"4,8,11","2,057,100",0,0
9,AMERICAN EXPRESS CO,COM,025816109,"673,468","7,994,634",SH,,DFND,"4,10","7,994,634",0,0
